In [ ]:
# Step 1 -- See if being dumb works.
# Spoilers -- it didn't work for Star #2.
# Step 2 -- Don't be dumb.
# it's just a clever setup for CRT, but we will still write a
# generalized solution... which ended up being way more complicated
# than necessary.

from collections import defaultdict
from functools import reduce
import itertools


class network:
    def __init__(self, name: str, nodes=None, edges=None, path=None) -> None:
        self.name: str = name
        self.nodes: set[str] = set(nodes) if nodes else set()
        self.edges: list[tuple[str, str]] = edges if edges else []
        self.edge_by_node: defaultdict[str, list[str]] = defaultdict(list)
        # Should really minimize this path first... but I am lazy.
        self.path = path if path else []

        for _edge in self.edges:
            self.nodes.update([_edge[0], _edge[1]])
            self.edge_by_node[_edge[0]].append(_edge[1])

    def __str__(self) -> str:
        return f"""
            ----------------------
            Network {self.name} with {len(self.nodes)} nodes
            and {len(self.edges)} edges.
            ----------------------
            Nodes: {self.nodes}
            ----------------------
            Edges: {self.edges}
            ----------------------
            Edge by node: {self.edge_by_node}
            ----------------------
            Path: {self.path}
            ----------------------
            """

    def __repr__(self) -> str:
        return self.__str__()

    def add_edge(self, a_edge: tuple[str, str]) -> None:
        self.edges.append(a_edge)
        self.nodes.update([a_edge[0], a_edge[1]])
        self.edge_by_node[a_edge[0]].append(a_edge[1])

    def add_node(self, a_node: str) -> None:
        self.nodes.append(a_node)

    # Given a node, follow the path of the network and return the
    # nodes that are visited that end in Z and the end node.
    def follow_path(self, fp_node: str, full: bool = False) -> list[(int, str)]:
        l: list[(int, str)] = []
        for index, p in enumerate(self.path):
            fp_node = self.edge_by_node[fp_node][p]
            if not full and fp_node[-1] == "Z":
                l.append((index, fp_node))
            if full:
                l.append((index, fp_node))
        if not full and fp_node[-1] != "Z":
            l.append((len(self.path)-1, fp_node))
        return l

    def find_cycle(self, fc_start_node: str) -> tuple[list[str], int]:
        _tortoise: list[str] = [fc_start_node]
        _hare: list[str] = [fc_start_node,
                            self.edge_by_node[fc_start_node][self.path[0]]]
        _tortoise_index: int = 0
        _hare_index: int = 1 % len(self.path)
        _check_path: list[str] = [fc_start_node]
        while len(_check_path) == 1 or _check_path[-1] != fc_start_node:
            _check_path.extend(
                [x[1] for x in self.follow_path(_check_path[-1], full=True)])
        _check_path = _check_path[:-1]
        while True:
            if _hare == _tortoise*2:
                print("Found cycle for " + fc_start_node +
                      " of length " + f"{len(_tortoise)}")
                # Double Check
                if _check_path*len(_tortoise) == _tortoise*len(_check_path):
                    break
                print("False alarm, continuing...")
                print(f"{_tortoise[0], _tortoise[-1]}" +
                      f"{fc_start_node, _check_path[-1]}")
            _tortoise.append(
                self.edge_by_node[_tortoise[-1]][self.path[_tortoise_index]])
            _tortoise_index = (_tortoise_index + 1) % len(self.path)
            while len(_hare) < 2*len(_tortoise):
                _hare.append(
                    self.edge_by_node[_hare[-1]][self.path[_hare_index]])
                _hare_index = (_hare_index + 1) % len(self.path)
        return ([(index, _tortoise[index]) for index in range(len(_tortoise)) if _tortoise[index][-1] == "Z"], len(_tortoise))

    def find_valid_indicies(self, fvi_node: str) -> tuple[list[tuple[int, str, bool]], int]:
        fvi_front_tail: list[(int, str)] = []
        seen: set[str] = set()
        seen_with_index: list[(int, str)] = []
        fvi_num_iterations: int = 0
        while fvi_node not in seen:
            seen.add(fvi_node)
            _zs: list[(int, str)] = self.follow_path(fvi_node)
            fvi_front_tail.extend([(z[0]+fvi_num_iterations*len(self.path), z[1])
                                   for z in _zs])
            fvi_node = _zs[-1][1]
            seen_with_index.append(_zs[-1])
            fvi_num_iterations += 1
        cycle_start_index: int = -1
        for ix, node in seen_with_index:
            if node == fvi_node:
                cycle_start_index = ix
                break
        cycle_length = self.find_cycle(fvi_node)[1]
        r = []
        for index, node in fvi_front_tail:
            if index - cycle_start_index >= cycle_length:
                break
            if node[-1] == "Z":
                r.append(
                    (index, node, index >= cycle_start_index))
        rtn_val: tuple[list[tuple[int, str, bool]],
                       int] = tuple([r, cycle_length])
        print(rtn_val)
        return rtn_val

    def ghost_walk(self) -> list[int]:
        # Oh no-- we've gone too far.
        gw_find_common: list[tuple[list[tuple[int, str, bool]], int]] = []
        for gw_node in self.nodes:
            if gw_node[-1] == "A":
                gw_find_common.append(self.find_valid_indicies(gw_node))
        short_list: set[int] = set()
        rem_and_mods: list[list[tuple[int, int]]] = []
        for _x in gw_find_common:
            rem_and_mods.append([])
            for _node in _x[0]:
                if not _node[2]:
                    short_list.add(_node[0])
                    continue
                print(f"x = {_node[0]} mod {_x[1]}")
                rem_and_mods[-1].append((_node[0], _x[1]))
        for _x in sorted(list(short_list)):
            if all([any([_x == _z[0] for _z in _y]) for _y, _ in gw_find_common]):
                return [_x+1]

        _slns: list[int] = []
        for _sln_set in itertools.product(*rem_and_mods):
            cur_m_r: tuple[int, int] = _sln_set[0]
            for mr in _sln_set[1:]:
                cur_m_r = crt(cur_m_r, mr)
            _slns.append(cur_m_r[0]+1)
        return _slns


def bezout(a, b):
    u1, v1, r1 = 1, 0, a
    u2, v2, r2 = 0, 1, b

    while r2:
        q = r1 // r2
        u1, u2 = u2, u1-q*u2
        v1, v2 = v2, v1-q*v2
        r1, r2 = r2, r1-q*r2
        assert u1*a + v1*b == r1
        assert u2*a + v2*b == r2

    if r1 < 0:
        u1, v1, r1 = -u1, -v1, -r1

    # a_coefficient, b_coefficient, gcd
    return (u1, v1, r1)


def crt(cong1, cong2):
    a1, n1 = cong1
    a2, n2 = cong2
    c1, c2, g = bezout(n1, n2)
    assert n1*c1 + n2*c2 == g

    if (a1 - a2) % g != 0:
        raise ValueError(f"Incompatible congruences {cong1} and {cong2}.")

    lcm = n1 // g * n2
    rem = (a1*c2*n2 + a2*c1*n1)//g
    return rem % lcm, lcm


def parse(p_name: str, lines: list[str]) -> network:
    # First line is the LR instructions.
    _lr: list[int] = []
    _edges: list[tuple[str, str]] = []

    for _ix, _line in enumerate(lines):
        if _ix == 0:
            _lr = list(map(lambda x: 0 if x == "L" else 1, _line.strip()))
            continue
        if len(_line) == 0:
            continue
        _n, _e = _line.split("=")
        _n = _n.strip()
        _e = list(map(lambda x: x.strip(), _e.strip().strip("()").split(",")))
        for _ix, _edge in enumerate(_e):
            _edges.append(tuple([_n, _edge, _ix]))
    _n = network(name=p_name, edges=_edges, path=_lr)
    return _n

In [ ]:
fn = "day8.txt"
with open("input_data/"+fn, "r") as f:
    contents = list(filter(lambda x: len(x) > 0, map(
        lambda x: x.strip(), f.readlines())))
    net: network = parse(p_name=fn, lines=contents)
    sln: list[int] = net.ghost_walk()
    print(sln)